# پروژه برنامه ریزی خطی
نام نام خانوادگی: کیوان خوجم لی - 
نام درس : الگوریتم های پیشرفته -
شماره دانشجویی : 401422079 

## شروع پروژه وتوضیحات لازم

## مقدمه
پروژه ایی که در زیر مشاهده میکنید یک برنامه به زبان پایتون است و از برنامه ریزی خطی استفاده میکند تا یک برنامه رژیمی برای یک روز یک نفر که دچار یک محدودیت است بنویسد.(ایده حل برای یک هفته میتواند لیست تابو باشد!) از آنجا که محدودیت زمانی و داده ایی بود برنامه فقط برای 3 نوع محدودیت نوشته شده و از آنجا که محدودیت واضح است ترجیچات غذایی را حذف کرده وبر اساس نیاز طبیعی  و محدودیت شخص برنامه زیر نوشته شده است.
محدودیت ها شامل دیابت و پوکی استخوان و چربی خون بالا میشود!

نکته : تمام اطلاعات استفاده شده در این پروژه صرفا از طریق سرچ در گوگل به دست آمده و ارزش علمی ندارد زیرا برای یه رژیم عوامل زیادی را باید در نظر بگیریم.

### بخش 1  ( پیش پردازش داده )

برای شروع کتابخانه های لازم را import  کردیم
از کتابخانه pulp  برای نوشتن معادلات بهینه خطیمون استفاده کردیم
سپس داده اولیه مواد غذایی را میخوانیم و سپس یه head  از داده ها میگیریم

In [1]:
from pulp import *
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'USDA.csv')
df.head()

,ID,Description,Calories,Protein,TotalFat,Carbohydrate,Sodium,SaturatedFat,Cholesterol,Sugar,Calcium,Iron,Potassium,VitaminC,VitaminE,VitaminD
0,1001,"BUTTER,WITH SALT",717.0,0.85,81.11,0.06,714.0,51.368,215.0,0.06,24.0,0.02,24.0,0.0,2.32,1.5
1,1002,"BUTTER,WHIPPED,WITH SALT",717.0,0.85,81.11,0.06,827.0,50.489,219.0,0.06,24.0,0.16,26.0,0.0,2.32,1.5
2,1003,"BUTTER OIL,ANHYDROUS",876.0,0.28,99.48,0.00,2.0,61.924,256.0,0.00,4.0,0.00,5.0,0.0,2.80,1.8
3,1004,"CHEESE,BLUE",353.0,21.40,28.74,2.34,1395.0,18.669,75.0,0.50,528.0,0.31,256.0,0.0,0.25,0.5
4,1005,"CHEESE,BRICK",371.0,23.24,29.68,2.79,560.0,18.764,94.0,0.51,674.0,0.43,136.0,0.0,0.26,0.5


اگه دقت کنید تعداد ردیف ها بسیار زیاد است و تقریبا چندین مدل از یک نوع ماده غذا داریم. در این بخش برای سبک تر شدن داده و معادلات ردیف هایی که تکراری هستند رو پاک کردیم و فقط دو نمونه از آنها را نگه داری کردیم تاتعداد قابل توجهی از متغییر های بهینه سازی خطی امان کم شود .و در سلول بعدی مشاهده کردیم مقادیر نال در ردیف های مختلف وجود دارد برای همین در سلول بعدی آن مقادیر نال را حذف میکنیم. 

In [3]:
def extract_text(text):
    return text.split(',')[0]
df["newDescription"]=df['Description'].apply(extract_text)
df=df[df.groupby("newDescription").cumcount()<2]


In [4]:
df.isnull().sum()

ID                  0
Description         0
Calories            0
Protein             0
TotalFat            0
Carbohydrate        0
Sodium             27
SaturatedFat      135
Cholesterol        98
Sugar             524
Calcium            28
Iron               21
Potassium         115
VitaminC           94
VitaminE          798
VitaminD          664
newDescription      0
dtype: int64

In [5]:

df.dropna(axis=0,inplace = True)
df = df[df["Calories"]>30]
df.reset_index(drop = True , inplace = True)


با توجه به اینکه بعضی از این ستون ها در کل برنامه به کار ما نمیآید  تعدادی از ستون هارا حذف کردیم تا کار ما محدود تر و محاسبات آسان تر بشود

In [6]:
df.drop("ID",axis = 1,inplace = True)
df.drop("Iron",axis = 1,inplace = True)
df.drop("TotalFat",axis = 1,inplace = True)
df.drop("Sodium",axis = 1,inplace = True)
df.drop("Potassium",axis = 1,inplace = True)
df.drop("Cholesterol",axis = 1,inplace = True)

با توجه به اینکه محدودیت اعمالی بنده برای هر ماده غذایی 150 گرم بود و ارزش غذایی مواد داخل دیتا فریم به ازای 100 گرم تنظیم شده بود آنها را  تبدیل به ازای هر 25 گرم میکنیم با تقسیم کردن هر ردیف عددی به 4

In [7]:
df['Calories']= df['Calories'].apply(lambda X : X/4)
df['Protein']= df['Protein'].apply(lambda X : X/4)
df['Carbohydrate']= df['Carbohydrate'].apply(lambda X : X/4)
df['SaturatedFat']= df['SaturatedFat'].apply(lambda X : X/4)
df['Sugar']= df['Sugar'].apply(lambda X : X/4)
df['Calcium']= df['Calcium'].apply(lambda X : X/4)
df['VitaminC']= df['VitaminC'].apply(lambda X : X/4)
df['VitaminE']= df['VitaminE'].apply(lambda X : X/4)
df['VitaminD']= df['VitaminD'].apply(lambda X : X/4)

In [8]:
df.head(10)

,Description,Calories,Protein,Carbohydrate,SaturatedFat,Sugar,Calcium,VitaminC,VitaminE,VitaminD,newDescription
0,"BUTTER,WITH SALT",179.25,0.2125,0.0150,12.84200,0.0150,6.00,0.000,0.5800,0.375,BUTTER
1,"BUTTER,WHIPPED,WITH SALT",179.25,0.2125,0.0150,12.62225,0.0150,6.00,0.000,0.5800,0.375,BUTTER
2,"BUTTER OIL,ANHYDROUS",219.00,0.0700,0.0000,15.48100,0.0000,1.00,0.000,0.7000,0.450,BUTTER OIL
3,"CHEESE,BLUE",88.25,5.3500,0.5850,4.66725,0.1250,132.00,0.000,0.0625,0.125,CHEESE
4,"CHEESE,BRICK",92.75,5.8100,0.6975,4.69100,0.1275,168.50,0.000,0.0650,0.125,CHEESE
5,"CHEESE FD,PAST PROCESS,AMERICAN,VITAMIN D FORT",82.50,4.2150,2.1400,3.76425,1.3975,170.50,0.000,0.1575,0.625,CHEESE FD
6,"CHEESE SPRD,PAST PROCESS,AMERICAN",72.50,4.1025,2.1825,3.33175,1.8300,140.50,0.000,0.0475,0.100,CHEESE SPRD
7,"CREAM,FLUID,HALF AND HALF",32.50,0.7400,1.0750,1.78950,0.0400,26.25,0.225,0.0825,0.050,CREAM
8,"CREAM,FLUID,LT (COFFEE CRM OR TABLE CRM)",48.75,0.6750,0.9150,3.00500,0.0350,24.00,0.200,0.1375,0.100,CREAM
9,EGGNOG,22.00,1.1375,2.0125,0.64775,2.0125,32.50,0.375,0.0525,0.300,EGGNOG


## بخش 2
در اینجا تابع رژیم غذایی را تعریف میکنیم که سه تا ورودی میگیرد ورودی اول یک دیکشنری است از محدودیت ها و نیاز طبیعی شخص حاصل میشود (مثلا داخل آن میزان کالری مورد نیاز گنجانده شده) ورودی دوم جدول داده های ما هست که به تابع میدهیم و در آخر یک عدد صحیح مثبت بین 1 2 3 است که نشان دهنده یکی از3 حالت خاص و محدودیت کاربران که در بخش مقدمه گفتیم نشات میگیرد. 

Part 1 : در این بخش شروع میکنیم پیاده سازی اولیه pulp  را انجام دهیم تا معادله ها را بسازیم برای حالت های خاص 1و 2. خب مسعله در این دو حالت یک مسعله بیشینه سازی است و آن هم بیشینه کردن است بیشینه کردن تعداد اقلام مصرفی کران بالا برای هر متعییر را 6 قرار میدهیم یعنی که کاربر میتواند تا 150 گرم از هر ماده غذایی را در طول روز مصرف کند و معادله بیشینه سازی را اضافه میکنیم  total_Variety معادله بیشینه سازی ما هستش. در ادامه به اعمال قید ها میپردازیم با توجه به dicdiet 

part 2 : در این بخش تفاوت در این است حالت خاص 3 که یعنی چربی خون بالا را مورد بررسی قرار میدهد را معادله بندی میکنیم در اینجا معاده از نو مینیمم سازی است و آن هم مینیمم کردن میزان چربی اشباع شده در بدن است یه نکته که مهم بود این است که کلسترول دریافتی از غذا تاثیر حاصی بر کلسترول خون نمیگذارد و تمام آن از طریق همین چربی اشباع شده است

part 3 : در این بخش فاینال که تمام معادلات اضافه شدن و حل صورت میگیرد و سپس خروجی های غیر صفر به صورت یک دیکشنری برگشت داده میشود!

In [9]:
def creatdiet(dicdiet,dataframe ,specialDiettaryNeeds):

#-------------------------****   PART 1   ****---------------------------------------------------------------------------
    if specialDiettaryNeeds !=3:
        prob = pulp.LpProblem("ALotOfVariety",LpMaximize)
        decision_variables=[]
        for rownum , row in dataframe.iterrows():
            variable = str("x"+str(rownum))
            variable = pulp.LpVariable(str(variable),lowBound= 0,upBound=6,cat = "Integer")
            decision_variables.append(variable)
        
        total_Variety = ""
        for formula in decision_variables:
            total_Variety+=formula
        
        prob += total_Variety
        #+++++++++++++++++++++++++++++++++++++++++   
        totall_calories=""
        for rownum , row in df.iterrows():
            for i , schedule in enumerate(decision_variables):
                if rownum == i :
                    formula = row["Calories"]*schedule
                    totall_calories += formula
        
        lowerlimit = dicdiet["Calories"]-200
        prob+=(totall_calories<=dicdiet["Calories"])
        prob+=(totall_calories>=lowerlimit)
        
        #+++++++++++++++++++++++++++++++++++++   
        
        if dicdiet['Sugar'] != None :    
            totall_Sugar=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Sugar"]*schedule
                        totall_Sugar += formula
            
            prob+=(totall_Sugar <= dicdiet["Sugar"])
            
        
        #+++++++++++++++++++++++++++++++++++++   
                
        
        if dicdiet['Carbohydrate'] != None :
            
            totall_Carbohydrate=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Carbohydrate"]*schedule
                        totall_Carbohydrate += formula
            
            prob+=(totall_Carbohydrate <= dicdiet["Carbohydrate"])
            
        #+++++++++++++++++++++++++++++++++++++   
                
        if dicdiet['SaturatedFat'] != None :
            
            totall_SaturatedFat=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["SaturatedFat"]*schedule
                        totall_SaturatedFat += formula
            
            prob+=(totall_SaturatedFat <= dicdiet["SaturatedFat"])
            
        #+++++++++++++++++++++++++++++++++++++   
        
        
        if dicdiet['VitaminE'] != None :
            
            totall_VitaminE=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminE"]*schedule
                        totall_VitaminE += formula
            
            prob+=(totall_VitaminE <= dicdiet["VitaminE"])
            
        #+++++++++++++++++++++++++++++++++++++   
                
        
        
        if dicdiet['VitaminC'] != None :
            
            totall_VitaminC=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminC"]*schedule
                        totall_VitaminC += formula
            
            prob+=(totall_VitaminC >= dicdiet["VitaminC"]) 
            
        #+++++++++++++++++++++++++++++++++++++   
        
        
        if dicdiet['Protein'] != None :
            
            totall_Protein=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Protein"]*schedule
                        totall_Protein += formula
            
            prob+=(totall_Protein >= dicdiet["Protein"]) 
            
        #+++++++++++++++++++++++++++++++++++++   
                
        
        if dicdiet['VitaminD'] != None :
            
            totall_VitaminD=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminD"]*schedule
                        totall_VitaminD += formula
            
            if isinstance(dicdiet["VitaminD"] , int) :
                prob+=(totall_VitaminD <= dicdiet["VitaminD"]) 
            else:
                prob+=(totall_VitaminD >= dicdiet["VitaminD"][0]) 
                prob+=(totall_VitaminD <= dicdiet["VitaminD"][1])
            
        #+++++++++++++++++++++++++++++++++++++   
            
        if dicdiet['Calcium'] != None :
            
            totall_Calcium=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Calcium"]*schedule
                        totall_Calcium += formula
            if isinstance(dicdiet["Calcium"] , int):
                prob+=(totall_Calcium <= dicdiet["Calcium"])
            else:
                prob+=(totall_Calcium >= dicdiet["Calcium"][0]) 
                prob+=(totall_Calcium <= dicdiet["Calcium"][1])
            
#-------------------------****   PART 2   ****---------------------------------------------------------------------------   
    else:
        prob = pulp.LpProblem("minimumSaturatedFat",LpMaximize)
        decision_variables=[]
        for rownum , row in dataframe.iterrows():
            variable = str("x"+str(rownum))
            variable = pulp.LpVariable(str(variable),lowBound= 0,upBound=6,cat = "Integer")
            decision_variables.append(variable)
        
        total_SaturatedFats=""
        for rownum , row in dataframe.iterrows():
            for i , schedule in enumerate(decision_variables):
                if rownum == i :
                    formula = row["SaturatedFat"]*schedule
                    total_SaturatedFats += formula
            
        prob+=total_SaturatedFats
        
        #+++++++++++++++++++++++++++++++++++++ 
        
        totall_calories=""
        for rownum , row in df.iterrows():
            for i , schedule in enumerate(decision_variables):
                if rownum == i :
                    formula = row["Calories"]*schedule
                    totall_calories += formula
        
        lowerlimit = dicdiet["Calories"]-200
        prob+=(totall_calories<=dicdiet["Calories"])
        prob+=(totall_calories>=lowerlimit)
        
        #+++++++++++++++++++++++++++++++++++++   
        
        if dicdiet['Sugar'] != None :    
            totall_Sugar=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Sugar"]*schedule
                        totall_Sugar += formula
            
            prob+=(totall_Sugar <= dicdiet["Sugar"])
            
        
        #+++++++++++++++++++++++++++++++++++++   
                
        
        if dicdiet['Carbohydrate'] != None :
            
            totall_Carbohydrate=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Carbohydrate"]*schedule
                        totall_Carbohydrate += formula
            
            prob+=(totall_Carbohydrate <= dicdiet["Carbohydrate"])
            
        #+++++++++++++++++++++++++++++++++++++  
        
         
        if dicdiet['VitaminE'] != None :
            
            totall_VitaminE=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminE"]*schedule
                        totall_VitaminE += formula
            
            prob+=(totall_VitaminE <= dicdiet["VitaminE"])
            
        #+++++++++++++++++++++++++++++++++++++   
        
        if dicdiet['VitaminC'] != None :
            
            totall_VitaminC=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminC"]*schedule
                        totall_VitaminC += formula
            
            prob+=(totall_VitaminC >= dicdiet["VitaminC"]) 
            
        #+++++++++++++++++++++++++++++++++++++   
        
        
        if dicdiet['Protein'] != None :
            
            totall_Protein=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Protein"]*schedule
                        totall_Protein += formula
            
            prob+=(totall_Protein >= dicdiet["Protein"]) 
            
        #+++++++++++++++++++++++++++++++++++++   
                
        
        if dicdiet['VitaminD'] != None :
            
            totall_VitaminD=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["VitaminD"]*schedule
                        totall_VitaminD += formula
            if isinstance(dicdiet["VitaminD"] , int) :
                prob+=(totall_VitaminD <= dicdiet["VitaminD"]) 
            else:
                prob+=(totall_VitaminD >= dicdiet["VitaminD"][0]) 
                prob+=(totall_VitaminD <= dicdiet["VitaminD"][1])
            
        #+++++++++++++++++++++++++++++++++++++   
            
        if dicdiet['Calcium'] != None :
            
            totall_Calcium=""
            for rownum , row in df.iterrows():
                for i , schedule in enumerate(decision_variables):
                    if rownum == i :
                        formula = row["Calcium"]*schedule
                        totall_Calcium += formula
            if isinstance(dicdiet["Calcium"] , int):
                prob+=(totall_Calcium <= dicdiet["Calcium"])
            else:
                prob+=(totall_Calcium >= dicdiet["Calcium"][0]) 
                prob+=(totall_Calcium <= dicdiet["Calcium"][1])
            
#-------------------------****   PART 3   ****---------------------------------------------------------------------------           


    optimizationResult = prob.solve()
    resultDic={}
    assert optimizationResult == LpStatusOptimal
    for v in prob.variables():
        if v.varValue!=0:
            resultDic[v.name]=v.varValue
        
            
    return resultDic
        

# بخش 3  (دریافت ورودی و تنظیم حالت های خاص مورد نیاز)

در این بخش ابتدا دیکشنری مواد مورد نیاز انسان به صورت دیفالت مطرح میشود و سپس از کاربر وزن وقد کاربر را میگیردو سپس پرهیز خاص کاربر را میگیرد وسپس BMI آن را محاسبه میکند و با توجه به آن  کالری حداکثری مورد نیاز آن  و حداقل پروتعین مورد نیاز آن بررسی میشود. 

Part 3 : در این بخش برای افراد دیابتی تغیین شده طبق تحقیقات بنده که درست هم نیست به احتمال خیلی بالا مصرف شکر و کربوهیدرات محدود باید بشود  والبته بد نیست میزان کالری هم کمتر شود.

part 4 : در این بخش برای افراد که پوکی استخوان دارند برنامه میچینیم با محدویت هایی که بیان میشود 

part 5 :در این بخش برای افراد که چربی خون بالا دارند برنامه میچینیم با محدویت هایی که بیان میشود 
 
part 6 : در این بخش تابع را صدا میزنیم که برنامه غذایی را بگیریم حال داده های جواب غیر صفر اند را جدا میکنیم از دیتا فریم ودیتا فریم جدید میسازیم و سپس نتایج را  در یک ستون ذخیره میکنیم تا ردیف با جواب آن کامل شود و سپس یک ردیف هم کالری پیشنهاد شده از آن غذا  در مقدار پیشنهادی میسازیم. سپس یک ستون که اضافه است را برمیداریم 

In [10]:
#default
dictForUser = {'Calories': 2200, 'Protein': None, 'Carbohydrate': 180, 'SaturatedFat': None,
       'Sugar': 36, 'Calcium': 1800, 'VitaminC': 50, 'VitaminE': 20, 'VitaminD': 25}

weight= float(input("enter your weight (Kg):"))
height= float(input("enter your height (cm):"))

print("1. diabetes")
print("2. Osteoprosis")
print("3. high blood fat")

while True :
    specialDiettaryNeeds=int(input("tell us which of the above restrictions you have (insert number):"))
    if specialDiettaryNeeds == 1 or specialDiettaryNeeds == 2 or specialDiettaryNeeds==3:
        break
BMI= weight / ((height /100) ** 2)

dictForUser['Calories']= 2200
dictForUser['Protein'] = weight * 0.8
print("********************************")
if BMI < 18.5 :
    dictForUser['Calories'] = 2500
elif BMI > 25 :
    dictForUser['Calories'] = 1700

    
#-------------------------****   PART 3   ****---------------------------------------------------------------------------   
if specialDiettaryNeeds == 1:
        dictForUser['Calories']=dictForUser['Calories'] -200
        dictForUser['Sugar'] = 12
        dictForUser['Carbohydrate'] = 50
        
        print("You should consume less than ",dictForUser['Sugar']," grams of sugar per day")
        
        print("You should consume less than ",dictForUser['Carbohydrate']," grams of carbohydrates per day")
        
        
        print("In addition to the above, you need about ",dictForUser['Calories']
              ," calories and a minimum of ",dictForUser['VitaminC']
              ," micrograms of vitamin C, a maximum of ",dictForUser['VitaminD']
              ," micrograms of vitamin D and a maximum of ",dictForUser['VitaminE']
              ," grams of vitamin E in your proper diet.")
        
#-------------------------****   PART 4   ****---------------------------------------------------------------------------      
elif specialDiettaryNeeds == 2:
    dictForUser['Protein']= weight
    dictForUser["VitaminD"]=[10,20]
    dictForUser["Calcium"]=[1000,2500]
    
    print("You should consume at least ",dictForUser['Protein']," grams of protein")
    print("You should consume about ",dictForUser["VitaminD"][0]," to ",dictForUser["VitaminD"][1]," micrograms of vitamin D")
    print("You should consume about ",dictForUser["Calcium"][0]," to ",dictForUser["Calcium"][1]," milligramS of Calcium ")
    
    print("In addition to the above, you need about ",dictForUser['Calories']
              ," calories and a minimum of ",dictForUser['VitaminC']
              ," micrograms of vitamin D and a maximum of ",dictForUser['VitaminE']
              ," grams of vitamin E in your proper diet.")
    
#-------------------------****   PART 5   ****---------------------------------------------------------------------------   
else:
    print("Your balanced diet should be such that saturated fat is the least in it, and cholesterol received from food is not very important.")
    print("In addition to the above, you need about ",dictForUser['Calories']
              ," calories and a minimum of ",dictForUser['VitaminC']
              ," micrograms of vitamin C, a maximum of ",dictForUser['VitaminD']
              ," micrograms of vitamin D and a maximum of ",dictForUser['VitaminE']
              ," grams of vitamin E in your proper diet.")
    
    
    
#-------------------------****   PART 6   ****---------------------------------------------------------------------------   
rawResualt = creatdiet(dictForUser , df , specialDiettaryNeeds)
semirawResualt={}
resualtlist=[]
resaultlistvalue=[]
for key,value in rawResualt.items():
    semirawResualt[key[1:]]=value
    resualtlist.append(int(key[1:]))
    resaultlistvalue.append(value)

df['index']=df.index
finalDf = df.iloc[resualtlist]
finalDf.reset_index(drop = True , inplace = True)
finalDf=finalDf.assign(resualt = resaultlistvalue)
finalDf["CaloriesConsumed"] = finalDf["Calories"]*finalDf["resualt"]
finalDf.drop("newDescription",axis = 1,inplace = True)

    

enter your weight (Kg):103
enter your height (cm):174
1. diabetes
2. Osteoprosis
3. high blood fat
tell us which of the above restrictions you have (insert number):1
********************************
You should consume less than  12  grams of sugar per day
You should consume less than  50  grams of carbohydrates per day
In addition to the above, you need about  1500  calories and a minimum of  50  micrograms of vitamin C, a maximum of  25  micrograms of vitamin D and a maximum of  20  grams of vitamin E in your proper diet.


 part 7 :حال لیست اقلام غذایی داریم حالا میتوانیم آنها را در چند وعده تقسیم کنیم میتوانیم یک چهارم را به شام وصبحانه اختصاص دهیم وبقیه را به نهار این تقسیم بندی را با استفاده از کالری به دست آمده انجام میدهیم و سپس اقلام صبحانه و نهار وشام را مینویسیم.
 
 part 8 :در این بخش میبینم چه میزان از آن بخش هایی که برای محدودیت اعمال شده در چه عددی قرار گرفته اند

In [11]:
#-------------------------****   PART 7   ****---------------------------------------------------------------------------   
finalCalorie=finalDf["CaloriesConsumed"].sum()
breakfastList=[]
breakfastListremining = finalCalorie/4
breakfastListCalorie=0
LunchList=[]
LunchListCalorie=0
DinnerList=[]
DinnerListCalorie=0
DinnerListremining = finalCalorie/4
for i in range(len(resualtlist)):
    if breakfastListremining > finalDf["CaloriesConsumed"][i] :
        breakfastList.append(i)
        breakfastListCalorie+=finalDf["CaloriesConsumed"][i]
        breakfastListremining-=finalDf["CaloriesConsumed"][i]
        
    elif DinnerListremining > finalDf["CaloriesConsumed"][i] :
        DinnerList.append(i)
        DinnerListCalorie+=finalDf["CaloriesConsumed"][i]
        DinnerListremining-=finalDf["CaloriesConsumed"][i]
        
    else:
        LunchList.append(i)
        LunchListCalorie+=finalDf["CaloriesConsumed"][i]

print("breakfast")
colomn_name = 'Description'
second_colomn_name= "resualt"
for row in breakfastList:
    value= finalDf[colomn_name][row]
    weighfood = finalDf[second_colomn_name][row]*25
    print( weighfood ,"grams", value)
    
print("breakfast List Calorie : ", breakfastListCalorie)

print('----------------')
print("Lunch")
colomn_name = 'Description'
second_colomn_name= "resualt"
for row in LunchList:
    value= finalDf[colomn_name][row]
    weighfood = finalDf[second_colomn_name][row]*25
    print( weighfood ,"grams", value)
    
print("Lunch List Calorie : ", LunchListCalorie)


print('----------------')

print("Dinner")
colomn_name = 'Description'
second_colomn_name= "resualt"
for row in DinnerList:
    value= finalDf[colomn_name][row]
    weighfood = finalDf[second_colomn_name][row]*25
    print( weighfood ,"grams", value)
    
print("Dinner List Calorie : ", DinnerListCalorie)

#-------------------------****   PART 8   ****---------------------------------------------------------------------------   

print('-------------------------------------')
if specialDiettaryNeeds == 1 :
    print("Calorie : ",finalCalorie)
    print('sugar : ',(finalDf["Sugar"]*finalDf["resualt"]).sum())
    print('Carbohydrate : ',(finalDf["Carbohydrate"]*finalDf["resualt"]).sum())

elif specialDiettaryNeeds == 2:
    print("Calorie : ",finalCalorie)
    print('Protein : ',(finalDf["Protein"]*finalDf["resualt"]).sum())
    print('VitaminD : ',(finalDf["VitaminD"]*finalDf["resualt"]).sum())
    print('Calcium : ',(finalDf["Calcium"]*finalDf["resualt"]).sum())
    
else :
    print("Calorie : ",finalCalorie)
    print('SaturatedFat : ',(finalDf["SaturatedFat"]*finalDf["resualt"]).sum())
    
        
    

breakfast
150.0 grams EGG,WHITE,RAW,FRESH
150.0 grams LAMBSQUARTERS,CKD,BLD,DRND,WO/SALT
50.0 grams OKRA,RAW
150.0 grams TURNIP GRNS&TURNIPS,FRZ,CKD,BLD,DRND,WO/SALT
150.0 grams EGG SUB,LIQ OR FRZ,FAT FREE
150.0 grams ALCOHOLIC BEV,BEER,REG,ALL
25.0 grams SCALLOP,MIXED SPECIES,RAW
50.0 grams SOYMILK (ALL FLAVORS),ENHANCED
breakfast List Calorie :  370.25
----------------
Lunch
150.0 grams POLLOCK,WALLEYE,RAW
150.0 grams LOBSTER,NORTHERN,RAW
150.0 grams SHRIMP,MIXED SPECIES,RAW
150.0 grams OYSTER,EASTERN,WILD,RAW
150.0 grams TOFU,FIRM,PREP W/CA SULFATE&MAGNESIUM CHLORIDE (NIGARI)
150.0 grams TOFU,SOFT,PREP W/CA SULFATE&MAGNESIUM CHLORIDE (NIGARI)
150.0 grams BEAN BEVERAGE
150.0 grams JELLYFISH,DRIED,SALTED
150.0 grams FROG LEGS,RAW
Lunch List Calorie :  793.5
----------------
Dinner
150.0 grams FLATFISH (FLOUNDER&SOLE SP),RAW
150.0 grams HADDOCK,RAW
150.0 grams OCEAN PERCH,ATLANTIC,RAW
Dinner List Calorie :  334.5
-------------------------------------
Calorie :  1498.25
sugar :  11.4649

In [12]:
finalDf.head(len(resualtlist)) #final data frame 

,Description,Calories,Protein,Carbohydrate,SaturatedFat,Sugar,Calcium,VitaminC,VitaminE,VitaminD,index,resualt,CaloriesConsumed
0,"EGG,WHITE,RAW,FRESH",13.00,2.7250,0.1825,0.00000,0.1775,1.75,0.000,0.0000,0.000,23,6.0,78.00
1,"LAMBSQUARTERS,CKD,BLD,DRND,WO/SALT",8.00,0.8000,1.2500,0.01300,0.1550,64.50,9.250,0.4625,0.000,321,6.0,48.00
2,"OKRA,RAW",7.75,0.5000,1.7575,0.00650,0.3000,20.25,5.275,0.0900,0.000,325,2.0,15.50
3,"TURNIP GRNS&TURNIPS,FRZ,CKD,BLD,DRND,WO/SALT",8.75,0.7475,1.2125,0.02125,0.2675,32.00,4.550,0.5325,0.000,357,6.0,52.50
4,"EGG SUB,LIQ OR FRZ,FAT FREE",12.00,2.5000,0.5000,0.00000,0.5000,18.25,0.125,0.3975,0.400,36,6.0,72.00
5,"ALCOHOLIC BEV,BEER,REG,ALL",10.75,0.1150,0.8875,0.00000,0.0000,1.00,0.000,0.0000,0.000,413,6.0,64.50
6,"FLATFISH (FLOUNDER&SOLE SP),RAW",17.50,3.1025,0.0000,0.11025,0.0000,5.25,0.000,0.1575,0.700,467,6.0,105.00
7,"HADDOCK,RAW",18.50,4.0800,0.0000,0.02275,0.0000,2.75,0.000,0.1125,0.125,469,6.0,111.00
8,"OCEAN PERCH,ATLANTIC,RAW",19.75,3.8275,0.0000,0.06825,0.0000,7.00,0.000,0.1900,0.300,477,6.0,118.50
9,"POLLOCK,WALLEYE,RAW",14.00,3.0475,0.0000,0.03100,0.0000,3.75,0.000,0.1225,0.050,482,6.0,84.00
